In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import pytz

# Import State change data

In [54]:
# From bpod (Need to edit for every new entry)
TrialStartTimeStamp = 0.5334
TrialEndTimeStamp = 1341.1 + 1

In [55]:
state_file_path = '/Users/zhangpan/MasterProject/Clive/Training_Data_New_Cohort1/New_TrainingDay8_01052024/State_Data_01052024_Training_Cohort1_Day8_Clive.csv'
pupil_file_path = '/Users/zhangpan/MasterProject/Clive/Training_Data_New_Cohort1/New_TrainingDay8_01052024/Pupil_Data_01052024_Training_Cohort1_Day8_Clive.csv'

In [56]:
df_change_state = pd.read_csv(state_file_path, header=None, names=['state_count', 'timestamp'])
# Convert the 'timestamp' column to pandas datetime format
df_change_state['timestamp'] = pd.to_datetime(df_change_state['timestamp'])

# Calculate the time difference in seconds from the first timestamp
df_change_state['seconds_since_start'] = (df_change_state['timestamp'] - df_change_state['timestamp'].iloc[0]).dt.total_seconds()
df_change_state['duration'] = df_change_state['seconds_since_start'].diff().shift(-1, fill_value=None)

# Add the time before the first state change (the first trial start timestamp)
df_change_state['seconds_since_start'] = df_change_state['seconds_since_start']+ TrialStartTimeStamp

# Insert the new row at the start of trial
new_row = {'timestamp': '0',
          'seconds_since_start':0
          }  # Replace with your desired timestamp

df_change_state.loc[-1] = new_row
df_change_state = df_change_state.sort_index().reset_index(drop=True)

# Add timestamp of trial start
df_change_state.at[0, 'timestamp'] = df_change_state.at[1, 'timestamp'] - timedelta(seconds=TrialStartTimeStamp)

# Get trial start time aligned from bonsai
trial_start_bonsai_time = df_change_state['timestamp'][0]
trial_start_bonsai_time

Timestamp('2024-05-01 11:50:41.562292800+0100', tz='pytz.FixedOffset(60)')

# Pupil size

In [57]:
df_pupil = pd.read_csv(pupil_file_path, header=None, names=['x', 'y', 'timestamp'])


# cut above the start of the trial (using bonsai time)
df_pupil['timestamp'] = pd.to_datetime(df_pupil['timestamp'])
df_pupil['seconds_since_start'] = (df_pupil['timestamp'] - trial_start_bonsai_time).dt.total_seconds()
df_pupil = df_pupil[df_pupil['seconds_since_start'] >= 0].reset_index(drop=True)

# cut below the end of last trial (using bpod time +1s just in case)
df_pupil_size = df_pupil[df_pupil['seconds_since_start'] <= TrialEndTimeStamp].reset_index(drop=True)
df_pupil_size = df_pupil_size.drop('timestamp', axis=1)
df_pupil_size = df_pupil_size.rename(columns={'seconds_since_start': 'timestamp'})
df_pupil_size

,x,y,timestamp
0,236.032822,103.321960,0.005963
1,236.851044,104.402084,0.021976
2,236.198654,104.218407,0.037285
3,236.018463,104.320221,0.053477
4,236.882248,103.949203,0.068965
...,...,...,...
81320,213.144226,98.713432,1342.027736
81321,212.043915,99.079849,1342.043595
81322,209.662979,103.547188,1342.059339
81323,209.135757,106.630547,1342.075058


In [58]:
df_pupil_size.to_csv('New_Pupil_Data_01052024_Day8_Clive.csv', index=False)